# Mosaic test
This notebook is used to test out cutting the larger complete map (made of large 1km * 1km mosaics) into smaller 500 * 500 pixel images.

- 1 tile = 1km * 1km
- 1 tile = 100.000cm * 100.000cm
- 100.000cm * 100.000cm / 8 cm per pixel = 12500 px * 12500 px
- (12500*12500)/(500*500) = 625
- final = 625 tiles of 500 8 500 pixels

# Imports

In [ ]:
import numpy as np
import xarray as xr
import xrspatial as xrs
import rioxarray as rxr

import stackstac as stc

tile_dir = "nl_8cm"
first_tile_dir = tile_dir + "/2022_128000_453000_RGB_hrl.tif"

# Reshaping
## Example of reshaping

In [ ]:
fake_tile = np.ones((12500, 12500))

In [ ]:
fake_tile.reshape((-1,500,500)).shape

## Reading first tile

In [ ]:
first_tile = rxr.open_rasterio(first_tile_dir)
first_tile = first_tile.rio.reproject("EPSG:28992")

In [ ]:
first_tile

In [ ]:
mos = first_tile.coarsen(x=500, y=500).construct(
    x=("x_mosaic", "x"), y=("y_mosaic", "y")
)

In [ ]:
first_tile.plot.imshow()